# Table of Content

* **Data preparation**
    * Load dataset
    * Split dataset
    
* **Deep Learning - CNN**
    * Data augmentation
    * Create CNN model
    * Model training

* **Evaluate**
    * History chart
    * Confusion Matrix
    * Performance test

* **Submission**


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preparation

## Load datasets

In [ ]:
train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')
submission_data = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [ ]:
print(f"train.csv size is {train_data.shape}")
print(f"test.csv size is {test_data.shape}")

In [ ]:
X = train_data.drop(['label'], axis=1).values
y = train_data['label'].values

In [ ]:
from keras.utils.np_utils import to_categorical

X = X.reshape(-1,28,28,1)
y = to_categorical(y)

## Split dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) #ratio 90:10

In [ ]:
print(f"Training data size is {X_train.shape}")
print(f"Testing data size is {X_test.shape}")

# Deep Learning - CNN

## Data augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    fill_mode='nearest',
    rescale=1./255,
)

In [ ]:
train_gen = datagen.flow(X_train, y_train, batch_size = 64)
test_gen = datagen.flow(X_test, y_test, batch_size = 64)

## Create CNN model

In [ ]:
import keras

model = keras.models.Sequential([
    keras.layers.Conv2D(16, (3,3), padding = 'same', activation='relu', input_shape=(28, 28, 1)),
    keras.layers.Conv2D(32, (3,3), padding = 'same', activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(32, (3,3), padding = 'same', activation='relu'),
    keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    keras.layers.Conv2D(256, (3,3), padding = 'same', activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.2),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

## Model training

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

es = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.0001,
    patience=10,
    verbose=1,
    mode='max',
    restore_best_weights=True
)

lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.2,
    patience=5,
    verbose=1,
    mode='max',
    min_delta=0.0001,
    min_lr=0.000001
)

In [ ]:
batch_size = 64
train_steps = len(X_train) / batch_size
test_steps = len(X_test) / batch_size

history = model.fit(
    train_gen,
    validation_data = test_gen,
    epochs = 100, 
    steps_per_epoch = train_steps,
    validation_steps = test_steps,
    callbacks=[es, lr],
    verbose=1
)

print("\n[INFO] Train Accuracy:", "{:.5f}%".format(history.history['accuracy'][-1]*100))
print("[INFO] Train Loss    :", "{:.7f}".format(history.history['loss'][-1]))
print("[INFO] Test Accuracy  :", "{:.5f}%".format(history.history['val_accuracy'][-1]*100))
print("[INFO] Test Loss      :", "{:.7f}".format(history.history['val_loss'][-1]))

# Evaluate

In [ ]:
evaluate_test = model.evaluate(X_test, y_test, verbose=1)

print("\nAccuracy =", "{:.7f}%".format(evaluate_test[1]*100))
print("Loss     =" ,"{:.9f}".format(evaluate_test[0]))

## History chart

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')

plt.show()

## Confusion matrix

In [ ]:
new_X_test = X_test.reshape(len(X_test),28,28,1) / 255.0
new_y_test = np.argmax(y_test, axis=-1)
yhat_test = np.argmax(model.predict(X_test), axis=-1)

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(new_y_test, yhat_test)
df_cm = pd.DataFrame(conf_matrix,
                     index = range(0,10),
                     columns = range(0,10))
plt.figure(figsize = (8,6)) #ratio 4:3
sn.heatmap(df_cm, cmap="YlGnBu", fmt="d", cbar=False, annot=True)

## Performance test

In [ ]:
fig = plt.figure(figsize=(15,20)) #ratio 3:4
for pick_random in range(1,26):
    roll_number = np.random.randint(0, len(X_test)-1)
    output_result = (new_y_test[roll_number] == yhat_test[roll_number])
    
    fig.add_subplot(5, 5, pick_random)
    plt.title(f"Real: {new_y_test[roll_number]}, Predicted: {yhat_test[roll_number]} \nResult: {output_result}")
    plt.imshow(new_X_test[roll_number], cmap="binary")

# Submission

In [ ]:
submission_label = np.argmax(model.predict(test_data.values.reshape(-1,28,28,1)), axis=-1)
submission_label = pd.Series(submission_label, name="Label")

image_id = pd.Series(range(1,len(test_data)+1))
image_id = pd.Series(image_id, name="ImageId")

In [ ]:
submission = pd.concat([image_id,submission_label],axis = 1)
submission.to_csv("digit_recognizer_submission.csv", index=False)